In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=30ff21c93ba432a085d06371c5a9f469d1460ddd599da818b2a6ff7adea75739
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/data/movielens_ratings.csv',inferSchema=True,header=True)

In [7]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
training,test = data.randomSplit([0.8,0.2])

In [13]:
als = ALS(maxIter = 5,regParam=0.01, userCol='userId',itemCol='movieId',ratingCol='rating')

In [14]:
model = als.fit(training)

In [15]:
predictions = model.transform(test)

In [16]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      3|   1.0|    26|   0.9877058|
|      4|   1.0|    12|-0.022597611|
|      0|   1.0|    22| -0.29777873|
|      5|   2.0|    22|  0.44990265|
|      3|   1.0|    13|   1.4858677|
|      4|   2.0|    13|   0.9233813|
|      5|   1.0|    13|   0.6749323|
|      6|   1.0|    13|   0.9948336|
|      1|   1.0|     6|  0.77609783|
|      6|   1.0|     6|   1.2292943|
|      2|   2.0|    20|   1.5620136|
|      6|   1.0|    20|   2.0121384|
|      6|   1.0|    15|  -0.6408614|
|      3|   1.0|     9| -0.33436203|
|      6|   1.0|    17|   1.4530723|
|      6|   1.0|     4|   1.5548472|
|      0|   1.0|    23|   2.9209566|
|      2|   1.0|    23|   1.7954814|
|      2|   1.0|    25|    1.216149|
|      1|   1.0|    14| -0.96954906|
+-------+------+------+------------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')
rmse= evaluator.evaluate(predictions)

In [18]:
print('RMSE')
print(rmse)

RMSE
1.9231850116029379


In [21]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [22]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     21|    11|
|     45|    11|
|     67|    11|
|     71|    11|
|     76|    11|
|     77|    11|
|     78|    11|
|     94|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     94|    11|  4.3556523|
|     77|    11|  2.8037949|
|     21|    11|  2.5023675|
|     13|    11|  2.2404501|
|     78|    11|   1.431344|
|     45|    11|  1.3935738|
|     71|    11|0.008991003|
|     67|    11|-0.34292442|
|     76|    11| -1.3636248|
+-------+------+-----------+

